In [ ]:
import kagglehub as kg

# Download latest version
path = kg.dataset_download("splcher/animefacedataset")

print("Path to dataset files:", path)

In [ ]:
images_path = '/root/.cache/kagglehub/datasets/splcher/animefacedataset/versions/3/images'
path = '/root/.cache/kagglehub/datasets/splcher/animefacedataset/versions/3'

In [ ]:
image_size = [247, 247]

In [ ]:
!pip install torch torchvision matplotlib numpy tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np

inception = InceptionV3(input_shape=image_size + [3] , weights='imagenet', include_top=False)

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

# Path to images
image_dir = "/root/.cache/kagglehub/datasets/splcher/animefacedataset/versions/3/images"

# Define transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to 64x64
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize([0.5], [0.5])  # Normalize between -1 and 1
])


class AnimeFaceDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith(".jpg") or f.endswith(".png")]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image


dataset = AnimeFaceDataset(image_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


print(f"Dataset size: {len(dataset)}")


Dataset size: 63565


In [ ]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, noise_dim=100):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.ConvTranspose2d(noise_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

generator = Generator().cuda()


In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


discriminator = Discriminator().cuda()


In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()


lr = 0.0002
beta1 = 0.5

optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerD = optim.Adam(discriminator.parameters(), lr=lr, betas=(beta1, 0.999))


In [ ]:
import numpy as np
from tqdm import tqdm


num_epochs = 50
noise_dim = 100
fixed_noise = torch.randn(64, noise_dim, 1, 1).cuda()

for epoch in range(num_epochs):
    for i, real_images in enumerate(tqdm(dataloader)):
        real_images = real_images.cuda()
        batch_size = real_images.size(0)

        # Labels
        real_labels = torch.ones(batch_size, 1).cuda()
        fake_labels = torch.zeros(batch_size, 1).cuda()

        ### Train Discriminator ###
        discriminator.zero_grad()


        real_outputs = discriminator(real_images).view(-1, 1)
        loss_real = criterion(real_outputs, real_labels)


        noise = torch.randn(batch_size, noise_dim, 1, 1).cuda()
        fake_images = generator(noise)
        fake_outputs = discriminator(fake_images.detach()).view(-1, 1)
        loss_fake = criterion(fake_outputs, fake_labels)


        loss_D = loss_real + loss_fake
        loss_D.backward()
        optimizerD.step()


        generator.zero_grad()


        fake_outputs = discriminator(fake_images).view(-1, 1)
        loss_G = criterion(fake_outputs, real_labels)

        loss_G.backward()
        optimizerG.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss_D: {loss_D.item():.4f}, Loss_G: {loss_G.item():.4f}")

    with torch.no_grad():
        sample_images = generator(fixed_noise).cpu()


torch.save(generator.state_dict(), "dcgan_generator.pth")
torch.save(discriminator.state_dict(), "dcgan_discriminator.pth")


NameError: name 'torch' is not defined

In [ ]:
import matplotlib.pyplot as plt
import torchvision.utils as vutils

generator.load_state_dict(torch.load("dcgan_generator.pth"))
generator.eval()


with torch.no_grad():
    noise = torch.randn(16, noise_dim, 1, 1).cuda()
    fake_images = generator(noise).cpu()


plt.figure(figsize=(8,8))
plt.axis("off")
plt.imshow(np.transpose(vutils.make_grid(fake_images, padding=2, normalize=True), (1,2,0)))
plt.show()


KeyboardInterrupt: 